In [ ]:
%load_ext autoreload
%autoreload 2

from url_normalize import url_normalize
from trafilatura import fetch_url
from trafilatura.core import bare_extraction
from meri.utils import setup_logging

import logging

setup_logging()

from trafilatura.downloads import DEFAULT_HEADERS
from meri.scraper import get_user_agent

# Change user agent to ours
DEFAULT_HEADERS['User-Agent'] = get_user_agent()

logging.getLogger("trafilatura").setLevel(logging.DEBUG)


In [ ]:
from ipywidgets import widgets

default_url = "https://www.iltalehti.fi/kotimaa/a/ee7e74fe-b0ea-455e-9a9c-bdfb0059b947"

url_wg = widgets.Text(
    value="",
    placeholder=default_url,
    description="URL:",
    disabled=False   
)

display(url_wg)


In [ ]:

url = url_wg.value or default_url

url = url_normalize(url)

downloaded = fetch_url(url)

document = bare_extraction(
    downloaded,
    url=url,
    include_formatting=True,
    include_comments=False,
    include_images=True,
    with_metadata=True,
    include_tables=True,
    include_links=True
)
#display(document)
from IPython.display import display_markdown
display_markdown(document['text'], raw=True)


In [ ]:
document

In [ ]:
# Build :class:`Article` object from retrieved document

from meri.abc import Article, ArticleMeta, article_url, LinkLabel
from meri.utils import detect_language

def build_article(document):
    return Article(
        meta=ArticleMeta(
            title=document['title'],
            language=document['language'] or detect_language(document['text']),
            authors=[] if not document['author'] else [document['author']],
        ),
        text=document['text'],
        urls=[
            article_url(document['url'], type=LinkLabel.LINK_CANONICAL),
            article_url(url, type=LinkLabel.LINK_MOVED),
        ],
    )

article = build_article(document)

In [ ]:
from meri.llm import extract_interest_groups
context_groups = extract_interest_groups(article)

In [ ]:
display(context_groups.model_dump())